<h1> Notebook: [ Week #06: Multi-Agent System ]

In this notebook, we will explore the essential principles of multi-agent systems and provide an overview of the crewAI framework.

In [2]:
# Update on 13-09-2024 to use ipython magic %pip rather than !pip for greater compatibility with VS Code
%pip install python-dotenv --quiet

%pip install crewai --quiet
%pip install crewai-tools --quiet

%pip install openai --quiet
%pip install tiktoken --quiet

%pip install langchain --quiet
%pip install langchain-openai --quiet
%pip install langchain-experimental --quiet

%pip install lolviz --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Common imports
import os
from dotenv import load_dotenv
import json
import lolviz

# Import the key CrewAI classes
from crewai import Agent, Task, Crew


---
---
<br>

# 1. A More Secure Way to store Credentials

<pre><code>
    <del>API_KEY = getpass("Enter your OpenAI API Key")</del>
    <del>client = OpenAI(api_key=API_KEY)</del>
</code></pre>

> ⚠️ The way to directly specify our API key in the notebook or script is dangerous.

- **Unauthorized Access**: 
    - If the notebook or script is shared or stored in a repository, anyone can use your API key to access the associated services, potentially leading to data breaches or abuse of the service.
- **Code Leakage**: 
    - If the code is accidentally leaked or published, your API key becomes compromised.
- **Hard to maintain**: 
    - Imagine you can a key that being shared by 10 applications. What happens if the key expires and you need to replace all the keys with the new one.

---
It’s safer to use environment variables or a **configuration file (.env)** that is not included in the version control to store sensitive information like API keys. This way, the keys remain private and are not exposed in the codebase.

1. **What is a `.env` File?**
   - A `.env` file is a simple text file used to store configuration settings, environment variables, and other key-value pairs related to a Python project.
   - It typically contains sensitive information such as API keys, database credentials, or configuration settings.
   - Unlike hardcoding these values directly into your source code, you store them in a `.env` file, which is loaded into the project's environment during runtime.
   - using `.env` files in Python ensures secure management of sensitive information and allows for flexible configuration across different environments. It's a best practice for maintaining security and scalability in your projects! 🛡️🐍

2. **Why Use a `.env` File?**
   - **Security**: By keeping sensitive data separate from your codebase, you reduce the risk of accidentally exposing it.
   - **Flexibility**: You can easily switch between different configurations (development, testing, production) by modifying the `.env` file.
   - **Organization**: It provides a systematic way to manage project-specific variables.

3. **How to Create and Use a `.env` File:**
   - **Step 1: Create the `.env` File**:
     - In your project directory, create a file named `.env` at the root level.
     - This is where you'll store your configuration settings and sensitive information.
   - **Step 2: Install the Module**:
     - Install the `python-dotenv` library using the following command:
       ```
       pip install python-dotenv
       ```
   - **Step 3: Store Sensitive Data in the `.env` File**:
     - Define key-value pairs in your `.env` file.
     - For example:
       ```
       MY_KEY = "your_secret_key_here"
       ```
     - You can have multiple API_KEYs or other information to be stored in the `.env` file.
     - As long as you use different keys to associate with the different values, you can retrieve them using the key later.
   - **Step 4: Access the `.env` File in Your Python Code**:
     - Import the `dotenv` module in your Python code.
     - Use `load_dotenv()` to load variables from the `.env` file.
     - Access the values using `os.getenv("KEY")` for each key-value pair defined in the `.env` file.
     - Example:
       ```python
       import os
       from dotenv import load_dotenv

       load_dotenv()
       print(os.getenv("KEY"))
       ```
   - **Step 5: Compile and Run the Code**:
     - When you run your Python program, it will utilize the loaded environment variables from the `.env` file.


---

Up to this point, we have been inserting our API Key directly in our notebook
```Python
import openai

openai.api_key  = <...>
```

Starting from this week, we will modify the code above to read the api_key from a .env file.
- Go to `Explorer` in VS Code (first icon on the left sidebar" to create an `.env` file in the **SAME folder where this jupyter notebook file is located**.
    
- Double click on the newly created file to edit the file
- Include the following lines in the .env file
    - `OPENAI_API_KEY="<insert_your_OpenAI_API_key_here_WITHOUT_the_angled_brackets>"`
    
    - `OPENAI_MODEL_NAME="gpt-4o-mini"`

- Save and close the file
- ⚠️ Take note that the filename should be `".env"` without the extension `.txt`
- ⚠️ On MacOS' Finder, this file is hidden by default, but visible from VS Code's `Explorer`

In [4]:
# Load the environment variables
# If the .env file is not found, the function will return `False
load_dotenv('.env')

True

In [5]:
# After loading the environment variables, we can access them using the `os` module,
# e.g., `os.getenv('OPENAI_API_KEY')`

# Code below prints out the truncated OPENAI_API_KEY
print(f'OPENAI_API_KEY = "{os.getenv('OPENAI_API_KEY')[:20]}"')

OPENAI_API_KEY = "sk-proj-vyXlBSr_ppBh"


In [6]:
# Print out the `OPENAI_MODEL_NAME`
# This is to set the default model used by the CrewAI Agents (otherwise it will use `gpt-4`, which is much more expensive)
# How do we know we need to use `OPENAI_MODEL_NAME`?
# Always refer to the documentation of the package you are using
# Packages like CrewAI, OpenAI, and others have their own conventions
# Even missing a single letter in `OPENAI_MODEL_NAME` will cause the package to throw an error
# Because the package is not able to find the environment variable

print(f'OPENAI_MODEL_NAME = "{os.getenv('OPENAI_MODEL_NAME')}"')

OPENAI_MODEL_NAME = "gpt-4o-mini"


In [7]:
# We can also set the environment variables directly using Python
# But this will only be available in the current session
# Extra: This line also serves as a safeguard for those that did not set "OPEN_MODEL_NAME" in the .env file.
os.environ['OPENAI_MODEL_NAME'] = "gpt-4o-mini"

---

- There 2 common way packages/service access the API Key


    - A) The API_KEY is read from the environment variables directly by the package/service

        - We just have to make sure the `.env` file is loaded as the environment variables

        - During development, this is often done by using `load_dotenv()` or equivalent approaches.

        - In actual deployment environment (e.g., on the server that hosts our app), the platforms will have their
        way of allowing developers to specify the environment variables, such as uploading the `.env` or enter the key-value pairs (the content of the .env file) directly. We will cover that in next two topics.
        ---
        
    - B) Require the API_KEY to be passed to the Python object (usually a Python Class or Method)
    
        ```Python
        # This cell shows how to use the API_KEY in setting up OpenAI's SDK

        from openai import OpenAI

        API_KEY = os.getenv("OPENAI_API_KEY")

        client = OpenAI(api_key=API_KEY)
        ```
        ---

---
---
<br>


# 2. Let's Start with a Simple Crew

In this first example, we will create a simple Crew with 2 Agents.

---

## 2.1 Creating Agents

An agent is an autonomous unit programmed to:

- Perform tasks
- Make decisions
- Communicate with other agents

> Think of an agent as a member of a team, with specific skills and a particular job to do. 
> 
> Agents can have different roles like 'Researcher', 'Writer', or 'Customer Support', each contributing to the overall goal of the crew.

| Parameter | Description                                                                                                 |
|-----------|-------------------------------------------------------------------------------------------------------------|
| `role`      | Defines the agent's function within the crew. It determines the kind of tasks the agent is best suited for. |
| `goal`      | The individual objective that the agent aims to achieve. It guides the agent's decision-making process.     |
| `backstory` | Provides context to the agent's role and goal, enriching the interaction and collaboration dynamics.        |
| `allow_delegation` | Agents can delegate tasks or questions to one another, ensuring that each task is handled by the most suitable agent. Default is True. |

Read more about `Agent` here: [https://docs.crewai.com/core-concepts/Agents/](https://docs.crewai.com/core-concepts/Agents/)

In [8]:
agent_planner = Agent(
    role="Content Planner",

    goal="Plan engaging and factually accurate content on {topic}",

    backstory="""You're working on planning a blog article about the topic: {topic}."
    You collect information that helps the audience learn something about the topic and make informed decisions."
    Your work is the basis for the Content Writer to write an article on this topic.""",

    allow_delegation=False, # we will explain more about this later

	verbose=True, # to allow the agent to print out the steps it is taking
)


agent_writer = writer = Agent(
    role="Content Writer",

    goal="Write insightful and factually accurate opinion piece about the topic: {topic}",

    backstory="""You're working on a writing a new opinion piece about the topic: {topic}.
    You base your writing on the work of the Content Planner, who provides an outline and relevant context about the topic.
    You follow the main objectives and direction of the outline as provide by the Content Planner.""",

    allow_delegation=False, # we will explain more about this later

    verbose=True, # to allow the agent to print out the steps it is taking
)

## 2.2 Creating Tasks

In the crewAI framework, tasks are specific assignments completed by agents. 
- They provide all necessary details for execution, such as a description, the agent responsible, required tools, and more, facilitating a wide range of action complexities.

- Tasks within crewAI can be collaborative, requiring multiple agents to work together. 

    - This is managed through the task properties and orchestrated by the Crew's process, enhancing teamwork and efficiency.

<br>

| Parameters      | Description                                                                            |
|-----------------|----------------------------------------------------------------------------------------|
| `description`     | A clear, concise statement of what the task entails.                                   |
| `agent`           | The agent responsible for the task, assigned either directly or by the crew's process. |
| `expected_output` | A detailed description of what the task's completion looks like.                       |

In [10]:
task_plan = Task(
    description="""\
    1. Prioritize the latest trends, key players, and noteworthy news on {topic}.
    2. Identify the target audience, considering "their interests and pain points.
    3. Develop a detailed content outline, including introduction, key points, and a call to action.""",

    expected_output="""\
    A comprehensive content plan document with an outline, audience analysis, SEO keywords, and resources.""",

    agent=agent_planner,
)

task_write = Task(
    description="""\
    1. Use the content plan to craft a compelling blog post on {topic} based on the target audience's interests.
    2. Sections/Subtitles are properly named in an engaging manner.
    3. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing conclusion.
    4. Proofread for grammatical errors and alignment the common style used in tech blogs.""",

    expected_output="""
    A well-written blog post "in markdown format, ready for publication, each section should have 2 or 3 paragraphs.""",

    agent=agent_writer,
)

---

## 2.3 Creating the Crew

A crew in crewAI represents a collaborative group of agents working together to achieve a set of tasks. Each crew defines the strategy for task execution, agent collaboration, and the overall workflow.

- Create your crew of Agents

- Pass the tasks to be performed by those agents.

    - ⚠️⚠️⚠️ **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the *order* of the **tasks** in the list *matters*.

<br>

| Parameters | Description                                 |
|------------|---------------------------------------------|
| tasks      | A list of tasks assigned to the crew.       |
| agents     | A list of agents that are part of the crew. |

In [11]:
crew = Crew(
    agents=[agent_planner, agent_writer],
    tasks=[task_plan, task_write],
    verbose=True
)

## 2.4 Running the Crew

Once your crew is assembled, initiate the workflow with the `kickoff()` method. This starts the execution process according to the defined process flow.

In [12]:
# Start the crew's task execution
result = crew.kickoff(inputs={"topic": "Large Language Models"})

# Agent: Content Planner
## Task:     1. Prioritize the latest trends, key players, and noteworthy news on Large Language Models.
    2. Identify the target audience, considering "their interests and pain points.
    3. Develop a detailed content outline, including introduction, key points, and a call to action.


# Agent: Content Planner
## Final Answer: 
---

**Content Plan Document: Large Language Models**

### 1. Introduction

Large Language Models (LLMs) are revolutionizing the field of artificial intelligence, particularly in natural language processing (NLP). These models are capable of understanding, generating, and interacting in human language with remarkable fluency and accuracy. This article will delve into the current landscape of LLMs, the latest trends, key players, noteworthy news, and how businesses and individuals can leverage these models.

### 2. Target Audience Analysis

#### Interests:
- Tech enthusiasts looking to stay updated on AI advancements
- Data scientists

## 2.5 Examine the Crew Output

The output of a crew in the crewAI framework is encapsulated within the `CrewOutput` class. Imagine this is a proprietary object type from the crewAI framework (unlike a dictionary or a list in native Python).
This class provides a structured way to access results of the crew's execution, including various formats such as raw strings, JSON, and Pydantic models.

<br>

| Parameters   | Type                     | Description                                                                                          |
|--------------|--------------------------|------------------------------------------------------------------------------------------------------|
| `raw`         | str                      | The raw output of the crew. This is the default format for the output.                               |
| `json_dict`    | Optional[Dict[str, Any]] | A dictionary representing the JSON output of the crew.                                               |
| `tasks_output` | List[TaskOutput]         | A list of TaskOutput objects, each representing the output of a task in the crew.                    |
| `token_usage`  | Dict[str, Any]           | A summary of token usage, providing insights into the language model's performance during execution. |
| `pydantic`     | Optional[BaseModel]      | A Pydantic model object representing the structured output of the crew. (Out of Scope Advanced Topic)                              |


For more details on `CrewOutput`, please refer to the documentation [https://docs.crewai.com/core-concepts/Crews/#crew-output](https://docs.crewai.com/core-concepts/Crews/#crew-output)


In [13]:
# Accessing the crew output
print(f"Raw Output: {result.raw}")
print(f"Token Usage: {result.token_usage}")
print("-----------------------------------------\n\n")
print(f"Tasks Output of Task 1: {result.tasks_output[0]}")
print("-----------------------------------------\n\n")
print(f"Tasks Output of Task 2: {result.tasks_output[1]}")

Raw Output: ```markdown
# Large Language Models: Transforming the Landscape of Artificial Intelligence

## Introduction

In recent years, Large Language Models (LLMs) have emerged as a groundbreaking advancement in the realm of artificial intelligence, particularly within natural language processing (NLP). These sophisticated models excel in understanding, generating, and interacting with human language at an unprecedented level of fluency and accuracy. As we delve into the current landscape of LLMs, we'll explore the latest trends, identify key players in the field, highlight noteworthy news, and discuss how businesses and individuals can harness the potential of these models.

## Latest Trends in Large Language Models

LLMs are continually scaling in size and efficiency, driven by the insatiable demand for more powerful AI tools. Recent trends show a shift towards larger architectures that not only process vast amounts of text data but also improve computational efficiency. By enhanc

In [14]:
# One of the key distinctions between a standard LLM API call and Multi-agent system
# is that the latter uses significnatly more tokens due to the multiple agents involved.
print(f"Token Usage: {result.token_usage}")

Token Usage: total_tokens=4857 prompt_tokens=2294 completion_tokens=2563 successful_requests=3


## 2.6 🐍 Let's All the Code into a Python Script

1. Use the VS Code "Explorer" to create a new Python script file named `topic_6_simple_crew.py`.
2. Copy and paste the code below into the new file and execute it.

- Notice that the output of the script is easier to read on the terminal window than here in the notebook.


>💡 If you're unsure how to do this, see the note *"Writing & Running Python Scripts"* under Topic 6.

In [ ]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew


# Change the working directory to the current file's directory
os.chdir(os.path.dirname(os.path.abspath(__file__)))
print(f"The current working directory is: {os.getcwd()}")

load_dotenv('.env')


agent_planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",

    backstory="""You're working on planning a blog article about the topic: {topic}."
    You collect information that helps the audience learn something about the topic and make informed decisions."
    Your work is the basis for the Content Writer to write an article on this topic.""",

    allow_delegation=False, 
	verbose=True, 
)

agent_writer = writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate opinion piece about the topic: {topic}",

    backstory="""You're working on a writing a new opinion piece about the topic: {topic}.
    You base your writing on the work of the Content Planner, who provides an outline and relevant context about the topic.
    You follow the main objectives and direction of the outline as provide by the Content Planner.""",

    allow_delegation=False, 
    verbose=True, 
)

task_plan = Task(
    description="""\
    1. Prioritize the latest trends, key players, and noteworthy news on {topic}.
    2. Identify the target audience, considering "their interests and pain points.
    3. Develop a detailed content outline, including introduction, key points, and a call to action.""",

    expected_output="""\
    A comprehensive content plan document with an outline, audience analysis, SEO keywords, and resources.""",
    agent=agent_planner,
)

task_write = Task(
    description="""\
    1. Use the content plan to craft a compelling blog post on {topic} based on the target audience's interests.
    2. Sections/Subtitles are properly named in an engaging manner.
    3. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing conclusion.
    4. Proofread for grammatical errors and alignment the common style used in tech blogs.""",

    expected_output="""
    A well-written blog post "in markdown format, ready for publication, each section should have 2 or 3 paragraphs.""",
    agent=agent_writer,
)


crew = Crew(
    agents=[agent_planner, agent_writer],
    tasks=[task_plan, task_write],
    verbose=True
)

result = crew.kickoff(inputs={"topic": "Large Language Models"})


print(f"Raw Output: {result.raw}")
print("-----------------------------------------\n\n")
print(f"Token Usage: {result.token_usage}")
print("-----------------------------------------\n\n")
print(f"Tasks Output of Task 1: {result.tasks_output[0]}")
print("-----------------------------------------\n\n")
print(f"Tasks Output of Task 2: {result.tasks_output[1]}")

🎉🎉🎉 Hurrah! We have successfully run a multi-agent system using Python script, which works more like a program than a notebook.

It may not look like much now, but this is an important start to build a more flexible and modular application. 🎉🎉🎉

---

# 3. Give Tools to the Agents
A tool in CrewAI is a skill or function that agents can utilize to perform various actions. 

Tools are pivotal in extending the capabilities of CrewAI agents, enabling them to undertake a broad spectrum of tasks and collaborate effectively. When building solutions with CrewAI, leverage both custom and existing tools to empower your agents and enhance the AI ecosystem

- Tools are essential for LLM Agents as they significantly enhance their capabilities. 

- They enable agents to perform a wide range of tasks, from web searching and data analysis to content generation and collaboration. 

- Tools also provide customizability, allowing developers to create or integrate specific functionalities tailored to their needs. 

> for more info about `Tools`, such as *the list of tools* or *how to create your own tool*, see [https://docs.crewai.com/core-concepts/Tools/#introduction](https://docs.crewai.com/core-concepts/Tools/#introduction)



💡 [ IMPORTANT ] ***Different Ways to Give Agents Tools***

- **Agent Level:**
    - The Agent can use the Tool(s) on any Task it performs.
- **Task Level:**
    - The Agent will only use the Tool(s) when performing that specific Task.
    

- **🔥 Note**: **Task Tools** override the **Agent Tools**.



## 3.1 Tools from CrewAI Framework

The `WebsiteSearchTool` that we will try out is designed to conduct **semantic searches** within the content of websites. 
- It aims to leverage Retrieval-Augmented Generation (RAG) to navigate and extract information from specified URLs efficiently. 
- This tool intends to offer flexibility, allowing users to perform searches across any website or focus on specific websites of interest. 

Reference: https://docs.crewai.com/tools/WebsiteSearchTool/

In [15]:
from crewai_tools import WebsiteSearchTool

# Create a new instance of the WebsiteSearchTool
# Set the base URL of a website, e.g., "https://example.com/", so that the tool can search for sub-pages on that website
tool_websearch = WebsiteSearchTool("https://abc-notes.data.tech.gov.sg/")

Inserting batches in chromadb: 100%|██████████| 1/1 [00:02<00:00,  2.22s/it]


In [16]:
# Let's try to just run the tool by itself
search_result = tool_websearch.run("Parameters of Large Language Models: Temperature, Top-K, Top-P, Max Tokens")

# The returned output is a string, because it will be used part of context for the LLM agent who use this tool
print(f"Data Type of the Search Result: {type(search_result)}")

# We can see that even though the URL is set to the base URL, the tool is able to search for sub-pages
print(f"Search Result: {search_result}")

Using Tool: Search in a specific website
Data Type of the Search Result: <class 'str'>
Search Result: Relevant Content:
model="gpt-4o-mini"): messages = [{"role": "user", "content": prompt}] response = client.chat.completions.create( model=model, messages=messages, temperature=0,   this is the degree of randomness of the model's output ) Copy ✦ The method can accept more parameters than we are using here. ✦ There are three essential parameters here that can directly affect the behaviour of the LLMs. They are: - Temperature - Top-P - Top-K (not available on OpenAI models) ✦ These parameters are common for other LLMs, including Open-Source Models For more details on client.chat.completion.create() method, visit the offcial API reference here Temperature ✦ In the context of Large Language Models (LLMs) like GPT3.5 or GPT-4o, “temperature” refers to a parameter that controls the randomness of the model’s predictions. When you set a high temperature, the model is more likely to produce vari

---

**Now we modify our multi-agent system to include the `WebsiteSearchTool`.**
1. Use this code here to modify the previous Python script to include the `tool_websearch` in the `agent_planner` agent
2. Run the script and observe the output


In [17]:

# Creating Agents
agent_planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",

    backstory="""You're working on planning a blog article about the topic: {topic}."
    You collect information that helps the audience learn something about the topic and make informed decisions."
    Your work is the basis for the Content Writer to write an article on this topic.
    Make the best use of the tools provided to gather the as much necessary information as possible.""", #<--  A new line here

    tools=[tool_websearch], #<--  This is the line that includes the tool
    allow_delegation=False, 
	verbose=True, 
)

agent_writer = writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate opinion piece about the topic: {topic}",

    backstory="""You're working on a writing a new opinion piece about the topic: {topic}.
    You base your writing on the work of the Content Planner, who provides an outline and relevant context about the topic.
    You follow the main objectives and direction of the outline as provide by the Content Planner.""",

    allow_delegation=False, 
    verbose=True, 
)

# Creating Tasks
task_plan = Task(
    description="""\
    1. Prioritize the latest trends, key players, and noteworthy news on {topic}.
    2. Identify the target audience, considering "their interests and pain points.
    3. Develop a detailed content outline, including introduction, key points, and a call to action.""",

    expected_output="""\
    A comprehensive content plan document with an outline, audience analysis, SEO keywords, and resources.""",
    agent=agent_planner,
)

task_write = Task(
    description="""\
    1. Use the content plan to craft a compelling blog post on {topic} based on the target audience's interests.
    2. Sections/Subtitles are properly named in an engaging manner.
    3. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing conclusion.
    4. Proofread for grammatical errors and alignment the common style used in tech blogs.""",

    expected_output="""
    A well-written blog post "in markdown format, ready for publication, each section should have 2 or 3 paragraphs.""",
    agent=agent_writer,
)

# Creating the Crew
crew = Crew(
    agents=[agent_planner, agent_writer],
    tasks=[task_plan, task_write],
    verbose=True
)

# Running the Crew
result = crew.kickoff(inputs={"topic": "Mastering the Parameters (Temperature, Top-K, Top-P) of Large Language Models"})

2024-09-24 02:09:42,153 - 25132 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Content Planner
## Task:     1. Prioritize the latest trends, key players, and noteworthy news on Mastering the Parameters (Temperature, Top-K, Top-P) of Large Language Models.
    2. Identify the target audience, considering "their interests and pain points.
    3. Develop a detailed content outline, including introduction, key points, and a call to action.


# Agent: Content Planner
## Thought: I need to collect relevant information on mastering the parameters (Temperature, Top-K, Top-P) of Large Language Models, focusing on the latest trends, key players, and noteworthy news. I'll start by searching the specific website data.tech.gov.sg for relevant content.
Thought: I should search for the three key parameters (Temperature, Top-K, Top-P) one by one to gather comprehensive information.
## Using tool: Search in a specific website
## Tool Input: 
{
  "search_query": "Temperature parameter in Large Language Models"
}
## Tool Output: 
Relevant Content:
model="gpt-4o-mini"): mes

---

## 3.2 Third-Party Tool

CrewAI's ability to support not only its native tools but also third-party tools from [`LangChain`](https://python.langchain.com/v0.2/docs/integrations/tools/) and [`LlamaIndex`](https://llamahub.ai/) offers significant advantages. 

- This flexibility allows users to leverage a broader range of functionalities and integrations, enhancing the overall versatility and capability of the platform. 

- Developers are not confined to the tools provided by CrewAI alone; they can seamlessly integrate and utilize the best tools available in the market, tailored to their specific needs.

<br>

 For more info on **how to use**:
- `LangChain` tools in CrewAI, see https://docs.crewai.com/core-concepts/Using-LangChain-Tools/
- `LlamaIndex` tools in CrewAI, see https://docs.crewai.com/core-concepts/Using-LlamaIndex-Tools 

---
In this subsection, we're going to try a more advanced example that uses `toolkits` (a suit of tools) from LangChain to create a tool that can manipulate and analyze tabular data by actually running Python code.

- This tool uses the `pandas` library to manipulate the data and the `ChatOpenAI` agent to run the code.

- Bear with us, while this is a bit more complex, but we think it's worth to include it here because the simplier examples (using a single `tool` from LangChain) are already well documented in CrewAI's documentation.

- The `toolkits` are usually much more powerful and can be used to achieve more complex tasks, but we have yet to come across a comprehensive documentation on how to incorporate them into CrewAI's agents.

- We will take a pause from the previous example, so that we can use a tabular data that is needed for the demonstration.

In [18]:

from langchain.agents import Tool
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
import pandas as pd


df = pd.read_csv("https://raw.githubusercontent.com/pandas-dev/pandas/main/doc/data/titanic.csv")

pandas_tool_agent = create_pandas_dataframe_agent(
    llm=ChatOpenAI(temperature=0, model='gpt-4o-mini'),
    df=df, 
    agent_type=AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True # <-- This is an "acknowledgement" that this can run potentially dangerous code
)

# Create the tool
pandas_tool = Tool(
    name="Manipulate and Analyze tabular data with Code",
    func=pandas_tool_agent.invoke, # <-- This is the function that will be called when the tool is run. Note that there is no `()` at the end
    description="Useful for search-based queries",
)

**Execute the cell below to see the output**.

- 💪🏼 You may challenge yourself to put these two cells (above and below) into a proper Python script and follow the best practice for writing Python scripts.

In [19]:

# Creating Agents
agent_data_analyst = Agent(
    role="Content Planner",
    goal="Analyze the data based on user query: {topic}",
    backstory="""You're the best data analyst.""",
    allow_delegation=False,
	verbose=True,
    tools=[pandas_tool],
)

task_analyze = Task(
    description="""\
    1. Understand the user query: {topic}.
    2. Use the tool to analyze the data based on the user query.
    3. Develop a comprehensive report based on the analysis.""",

    expected_output="""\
    A comprehensive analysis report that present the results using McKinsey's Pyramid Principle.""",

    agent=agent_data_analyst,
)

# Creating the Crew
crew = Crew(
    agents=[agent_data_analyst],
    tasks=[task_analyze],
    verbose=True
)

# Running the Crew
result = crew.kickoff(inputs={"topic": "What is the average age of passengers in the Titanic dataset?"})

2024-09-24 02:12:33,258 - 25132 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Content Planner
## Task:     1. Understand the user query: What is the average age of passengers in the Titanic dataset?.
    2. Use the tool to analyze the data based on the user query.
    3. Develop a comprehensive report based on the analysis.


# Agent: Content Planner
## Thought: The user query is clear: they want to know the average age of passengers in the Titanic dataset. To provide a comprehensive analysis report, I will:
1. Retrieve the Titanic dataset.
2. Calculate the average age of passengers.
3. Present the results using McKinsey's Pyramid Principle, which involves leading with the key takeaway, then supporting that with detailed analysis.
First, I'll retrieve the dataset and calculate the average age using the provided tool.
## Using tool: Manipulate and Analyze tabular data with Code
## Tool Input: 
{
  "tool_input": "import pandas as pd\ntitanic_data = pd.read_csv('titanic.csv')\naverage_age = titanic_data['Age'].mean()\naverage_age"
}
## Tool Output: 
{'inpu

---
---
<br>


# 4. Agent Design Patterns in CrewAI

We will discuss 3 important concepts in CrewAI that can help us to better design Multi-agent Systems 


**Delegation**
- `delegation` in CrewAI is crucial for optimizing multi-agent systems as it allows agents to assign tasks to other agents, leveraging specialized skills within the crew. 

- This capability enhances flexibility and efficiency, enabling agents to manage complex tasks beyond their immediate scope by breaking them down into smaller, manageable sub-tasks handled by the most suitable agents. 

- Delegation reduces bottlenecks and ensures that each part of a task is executed by an agent with the best skills for that specific job, significantly improving overall system performance and task accuracy.



**Human Input** 
- `human_input` adds an essential layer of adaptability and oversight in multi-agent orchestration, allowing humans to guide, adjust, or intervene when necessary. 

- This input is particularly important for decision-making in uncertain or complex scenarios where human judgment and contextual understanding are crucial. 

- Integrating human input helps bridge the gap between automation and human expertise, enhancing the system's reliability, accuracy, and ability to handle unexpected challenges. 

- It ensures that the system remains aligned with strategic goals and adapts to real-world complexities that agents alone might not fully comprehend.



**Processes** 
- `processes` are vital because they define the structured execution flow of tasks within a multi-agent system. 

- By specifying whether tasks are executed sequentially, in parallel, or conditionally, processes allow for precise control over how agents interact and collaborate. 

- This structured approach ensures that tasks are completed in the most efficient order, minimizes errors, and maximizes resource utilization. 

- Processes enable complex orchestration by managing dependencies between tasks, ensuring smooth transitions, and allowing for scalable and adaptable workflows as the complexity of the task increases.


<br>

> 📒 For more concepts in CrewAI that can be used to orchestrate the multi-agent system, refer to the "Core Concepts" (esp. Collaboration, Pipeline, Training, and Memory) in the official documentation https://docs.crewai.com/
--- 

## 4.1 More "Power" - Allow Delegation

- By not setting `allow_delegation=False`, `allow_delegation` takes its default value of being `True`.
- This means the agent *can* delegate its work to another agent which is better suited to do a particular task.

![](https://abc-notes.data.tech.gov.sg/resources/img/crewai-delegation.png)

In this example, we will add a new agent (`Research Analyst`) to the crew
The `Research Analyst` will be responsible for conducting in-depth research to support the Content Planner

💡💡💡 Notice that the tool is not passed as an argument to the `agent`, but to the `task`.
- See our note on the difference between agent tools and task tools.
- In short:
    - Tools at the task level offer **deterministic** usage, ensuring that the task’s goals are met precisely as designed every time.
    - Tools at the agent level offer **probabilistic** usage, allowing agents the freedom to use tools based on the context, which can introduce variability but also flexibility in execution.


In [20]:
from crewai_tools import WebsiteSearchTool

# Create a new instance of the WebsiteSearchTool
# Set the base URL of a website, e.g., "https://example.com/", so that the tool can search for sub-pages on that website
tool_websearch = WebsiteSearchTool("https://abc-notes.data.tech.gov.sg/")


# A New Agent (Research Analyst)
agent_researcher = Agent(
    role="Research Analyst",
    goal="Conduct in-depth research on the topic: {topic}",
    backstory="""You're working on conducting in-depth research on the topic: {topic}.
    You will provide the Content Planner with the latest trends, key players, and noteworthy news on the topic.
    Based on the request that Content Planner, you will provide additional insights and resources from reliable and credible websites.
    You have access to web search tools and other resources to gather the necessary information.
    """,
    allow_delegation=False,
    verbose=True,
)

# New Tasks (Research Task) to be performed by the Research Analyst
task_research = Task(
    description="""\
    1. Conduct in-depth research on the topic: {topic}.
    2. Provide the Content Planner with the latest trends, key players, and noteworthy news on the topic.
    3. Pprovide additional insights and resources to enhance the content plan.
    4. Include latest developmnents in the research report.""",


    expected_output="""\
    A detailed research report with the latest trends, key players, and noteworthy news on the topic.""",

    agent=agent_researcher,
    tools=[tool_websearch],
)


In [21]:
# Creating Agents (the two existing agents)
agent_planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="""You're working on planning a blog article about the topic: {topic}.
    You collect information that helps the audience learn something about the topic and make informed decisions.
    Your work is the basis for the Content Writer to write an article on this topic.""",
    # Make the best use of the tools provided to gather the as much necessary information as possible.""",  <-- This line is removed.

    # Also removed the tool for this agent
    # Take note that this agent is now capable of delegating tasks to other agents
    allow_delegation=True,
	verbose=True,
)

agent_writer = writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate opinion piece about the topic: {topic}",

    backstory="""You're working on a writing a new opinion piece about the topic: {topic}.
    You base your writing on the work of the Content Planner, who provides an outline and relevant context about the topic.
    You follow the main objectives and direction of the outline as provide by the Content Planner.""",

    allow_delegation=False, 
    verbose=True, 
)


# Creating Tasks
task_plan = Task(
    description="""\
    1. Prioritize the latest trends, key players, and noteworthy news on {topic}.
    2. Identify the target audience, considering "their interests and pain points.
    3. Develop a detailed content outline, including introduction, key points, and a call to action.""",

    expected_output="""\
    A comprehensive content plan document with an outline, audience analysis, SEO keywords, and resources.""",
    agent=agent_planner,
)

task_write = Task(
    description="""\
    1. Use the content plan to craft a compelling blog post on {topic} based on the target audience's interests.
    2. Sections/Subtitles are properly named in an engaging manner.
    3. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing conclusion.
    4. Proofread for grammatical errors and alignment the common style used in tech blogs.""",

    expected_output="""
    A well-written blog post "in markdown format, ready for publication, each section should have 2 or 3 paragraphs.""",
    agent=agent_writer,
)

# Creating the Crew
crew = Crew(
    agents=[agent_planner, agent_researcher, agent_writer],
    tasks=[task_plan, task_write], # Note that the research task is not included here
    verbose=True
)

# Running the Crew
result = crew.kickoff(inputs={"topic": "Mastering the Parameters (Temperature, Top-K, Top-P) of Large Language Models"})

2024-09-24 02:16:17,037 - 25132 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Content Planner
## Task:     1. Prioritize the latest trends, key players, and noteworthy news on Mastering the Parameters (Temperature, Top-K, Top-P) of Large Language Models.
    2. Identify the target audience, considering "their interests and pain points.
    3. Develop a detailed content outline, including introduction, key points, and a call to action.
# Agent: Research Analyst
## Task: Gather the latest trends, key players, and noteworthy news on mastering the parameters (Temperature, Top-K, Top-P) of large language models.


# Agent: Research Analyst
## Final Answer: 
### Mastering the Parameters (Temperature, Top-K, Top-P) of Large Language Models

**Introduction:**
Large Language Models (LLMs) such as GPT-3, developed by OpenAI, have revolutionized natural language processing. Central to their operation are parameters like Temperature, Top-K, and Top-P, which play crucial roles in controlling the randomness and creativity of the generated text. Understanding how to m

---

## 4.2 The Man-in-the-Middle


- Human input provides critical oversight and adaptability in multi-agent systems, enabling human intervention in complex decision-making and enhancing the system's reliability, accuracy, and alignment with strategic goals. 

- Having a **'man-in-the-middle'** further strengthens this approach by allowing real-time monitoring and control over agent interactions, ensuring that the flow of information and decisions remain transparent and adjustable at any point. 

- This integration bridges the gap between automation and human expertise, allowing the system to handle uncertainties and real-world challenges more effectively.

![](https://abc-notes.data.tech.gov.sg/resources/img/crewai-human-input.png)

- In this demonstration, we will continue to use the same code from the previous example, with modification to allow the agent to get inputs from human users.

- The key changes are:
    - The `task_write` task now has `human_input=True` to indicate that the agent will need human input.
    
    - The `task_write` has a new line in the description to indicate that the agent should check with human users for feedback.

<br>

💡 Note: 
- The Writer_Agent will prompt us on VS Code to provide any input/feedback. If no input, we can just press "enter" to let the agent to continue on the task.
- Otherwise, just type in your comment/feedback on the top bar in VS Code when being prompted.
- Yes, this is not obvious and less than ideal. And that's why we need user-friendly frontends for our app! We will look into that in Topic 7 & 8.

In [22]:

from crewai_tools import WebsiteSearchTool

# Create a new instance of the WebsiteSearchTool
# Set the base URL of a website, e.g., "https://example.com/", so that the tool can search for sub-pages on that website
tool_websearch = WebsiteSearchTool("https://abc-notes.data.tech.gov.sg/")


# Creating Agents 
agent_planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="""You're working on planning a blog article about the topic: {topic}.
    You collect information that helps the audience learn something about the topic and make informed decisions.
    Your work is the basis for the Content Writer to write an article on this topic.""",
    # Make the best use of the tools provided to gather the as much necessary information as possible.""",  <-- This line is removed.

    # Also removed the tool for this agent
    # Take note that this agent is now capable of delegating tasks to other agents
    allow_delegation=True,
	verbose=True,
)

agent_researcher = Agent(
    role="Research Analyst",
    goal="Conduct in-depth research on the topic: {topic}",
    backstory="""You're working on conducting in-depth research on the topic: {topic}.
    You will provide the Content Planner with the latest trends, key players, and noteworthy news on the topic.
    Based on the request that Content Planner, you will provide additional insights and resources from reliable and credible websites.
    You have access to web search tools and other resources to gather the necessary information.
    """,
    allow_delegation=False,
    verbose=True,
)

agent_writer = writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate opinion piece about the topic: {topic}",

    backstory="""You're working on a writing a new opinion piece about the topic: {topic}.
    You base your writing on the work of the Content Planner, who provides an outline and relevant context about the topic.
    You follow the main objectives and direction of the outline as provide by the Content Planner.""",

    allow_delegation=False, 
    verbose=True, 
)


# <---------------------------------- Creating Tasks ---------------------------------->
task_plan = Task(
    description="""\
    1. Prioritize the latest trends, key players, and noteworthy news on {topic}.
    2. Identify the target audience, considering "their interests and pain points.
    3. Develop a detailed content outline, including introduction, key points, and a call to action.""",

    expected_output="""\
    A comprehensive content plan document with an outline, audience analysis, SEO keywords, and resources.""",
    agent=agent_planner,
)

task_research = Task(
    description="""\
    1. Conduct in-depth research on the topic: {topic}.
    2. Provide the Content Planner with the latest trends, key players, and noteworthy news on the topic.
    3. Pprovide additional insights and resources to enhance the content plan.
    4. Include latest developmnents in the research report.""",


    expected_output="""\
    A detailed research report with the latest trends, key players, and noteworthy news on the topic.""",

    agent=agent_researcher,
    tools=[tool_websearch],
)

task_write = Task(
    description="""\
    1. Use the content plan to craft a compelling blog post on {topic} based on the target audience's interests.
    2. Sections/Subtitles are properly named in an engaging manner.
    3. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing conclusion.
    4. Proofread for grammatical errors and alignment the common style used in tech blogs.
    5. You MUST check with human users to get their feedback on the content, and incorporate the suggestion to revise the content.""",

    expected_output="""
    A well-written blog post "in markdown format, ready for publication, each section should have 2 or 3 paragraphs.""",
    agent=agent_writer,
    human_input=True
)


# <---------------------------------- Creating the Crew ---------------------------------->
crew = Crew(
    agents=[agent_planner, agent_researcher, agent_writer],
    tasks=[task_plan, task_write], # Note that the research task is not included here
    verbose=True
)

# <---------------------------------- Running the Crew ---------------------------------->
# Running the Crew
result = crew.kickoff(inputs={"topic": "Mastering the Parameters (Temperature, Top-K, Top-P) of Large Language Models"})



2024-09-07 01:17:14,899 - 8221069120 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-09-07 01:17:14][DEBUG]: == Working Agent: Content Planner
 [2024-09-07 01:17:14][INFO]: == Starting Task:     1. Prioritize the latest trends, key players, and noteworthy news on Mastering the Parameters (Temperature, Top-K, Top-P) of Large Language Models.
    2. Identify the target audience, considering "their interests and pain points.
    3. Develop a detailed content outline, including introduction, key points, and a call to action.


> Entering new CrewAgentExecutor chain...
I need to gather comprehensive information about the latest trends, key players, and noteworthy news regarding the parameters of large language models (LLMs), specifically focusing on Temperature, Top-K, and Top-P. Additionally, I need to identify the target audience for the blog article and their interests or pain points. Finally, I will develop a detailed content outline that includes an introduction, key points, and a call to action.

Action: Delegate work to coworker
Action Input: {
    "task": "Re

## 4.3 Design the Flow 

- `Processes` are crucial in multi-agent systems as they define the structured execution flow of tasks, enabling precise control over agent interactions and task collaboration. 
- By managing task order, dependencies, and execution modes (sequential, parallel, or conditional),processes ensure efficiency, minimize errors, and support scalable, adaptable workflows.

- **By default**, CrewAI uses `Sequencial Process` where the task execution follows the predefined order in the task list, with the output of one task serving as context for the next.
    - For example, `[ Planninng Task -> Research Task -> Writing Task ]`, `Writing Task` will only see the output from `Research Task` but not the output from `Planning Task`.

- To customize task context, utilize the context parameter in the Task class to specify outputs that should be used as context for subsequent tasks.


---


In this demo, we will focus on "Asynchronous" executation, which is highly relevant in many real-life use cases.

- We can define a task to be executed asynchronously. This means that the crew will not wait for it to be completed to continue with the next task. This is useful for tasks that take a long time to be completed, or that are not crucial for the next tasks to be performed.

- We can then use the context attribute to define in a future task that it should wait for the output of the asynchronous task to be completed.

- For this example, both the `Planning Task` and `Research Task` will be executeda at the same time. `Writing Task` will include the outputs from both the tasks as part of its context.



![](https://abc-notes.data.tech.gov.sg/resources/img/crewai-async.png)

In [7]:
# In this example, we will demonstrate how 

from crewai_tools import WebsiteSearchTool

# Create a new instance of the WebsiteSearchTool
# Set the base URL of a website, e.g., "https://example.com/", so that the tool can search for sub-pages on that website
tool_websearch = WebsiteSearchTool("https://abc-notes.data.tech.gov.sg/")


# Creating Agents 
agent_planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="""You're working on planning a blog article about the topic: {topic}.
    You collect information that helps the audience learn something about the topic and make informed decisions.""",
    # Your work is the basis for the Content Writer to write an article on this topic.""", <-- This line is removed.

    allow_delegation=False, # <-- This is now set to False
	verbose=True,
)

agent_researcher = Agent(
    role="Research Analyst",
    goal="Conduct in-depth research on the topic: {topic}",
    backstory="""You're working on conducting in-depth research on the topic: {topic}.
    You have access to web search tools and other resources to gather the necessary information.""",
    # You will provide the Content Planner with the latest trends, key players, and noteworthy news on the topic. <-- This line is removed.
    # Based on the request that Content Planner, you will provide additional insights and resources from reliable and credible websites.""", <-- This line is removed.
    allow_delegation=False,
    verbose=True,

)

agent_writer = writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate opinion piece about the topic: {topic}",

    backstory="""You're working on a writing a new opinion piece about the topic: {topic}.
    You base your writing on the outline from Content Planner and the research report from the Research Analyst.""", # <-- New line added
    # You base your writing on the work of the Content Planner, who provides an outline and relevant context about the topic.  <-- This line is removed.
    # You follow the main objectives and direction of the outline as provide by the Content Planner.""",  <-- This line is removed.

    allow_delegation=False, 
    verbose=True, 
)


# <---------------------------------- Creating Tasks ---------------------------------->
task_plan = Task(
    description="""\
    1. Prioritize the latest trends, key players, and noteworthy news on {topic}.
    2. Identify the target audience, considering "their interests and pain points.
    3. Develop a detailed content outline, including introduction, key points, and a call to action.""",

    expected_output="""\
    A comprehensive content plan document with an outline, audience analysis, SEO keywords, and resources.""",
    agent=agent_planner,

    async_execution=True # Will be executed asynchronously [NEW]
)

task_research = Task(
    description="""\
    1. Conduct in-depth research on the topic: {topic}.
    2. Provide the Content Planner with the latest trends, key players, and noteworthy news on the topic.
    3. Pprovide additional insights and resources to enhance the content plan.
    4. Include latest developmnents in the research report.""",


    expected_output="""\
    A detailed research report with the latest trends, key players, and noteworthy news on the topic.""",

    agent=agent_researcher,
    tools=[tool_websearch],

    async_execution=True # Will be executed asynchronously [NEW] 
)

task_write = Task(
    description="""\
    1. Use the content plan to craft a compelling blog post on {topic} based on the target audience's interests.
    2. Sections/Subtitles are properly named in an engaging manner.
    3. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing conclusion.
    4. Proofread for grammatical errors and alignment the common style used in tech blogs.
    5. You MUST check with human users to get their feedback on the content, and incorporate the suggestion to revise the content.""",

    expected_output="""
    A well-written blog post "in markdown format, ready for publication, each section should have 2 or 3 paragraphs.""",
    agent=agent_writer,
    # human_input=True, # Remove user input for this task [NEW]

    context=[task_plan, task_research], # Will wait for the output of the two tasks to be completed,
    output_file="article.md" # <-- This allows the output of this task to be saved to a file [NEW]
)


# <---------------------------------- Creating the Crew ---------------------------------->
crew = Crew(
    agents=[agent_planner, agent_researcher, agent_writer],
    tasks=[task_plan, task_research, task_write], 
    verbose=True
)

# <---------------------------------- Running the Crew ---------------------------------->
# Running the Crew
result = crew.kickoff(inputs={"topic": "Mastering the Parameters (Temperature, Top-K, Top-P) of Large Language Models"})



2024-09-07 10:50:03,683 - 8660111168 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-09-07 10:50:03][DEBUG]: == Working Agent: Content Planner
 [2024-09-07 10:50:03][INFO]: == Starting Task:     1. Prioritize the latest trends, key players, and noteworthy news on Mastering the Parameters (Temperature, Top-K, Top-P) of Large Language Models.
    2. Identify the target audience, considering "their interests and pain points.
    3. Develop a detailed content outline, including introduction, key points, and a call to action.
 [2024-09-07 10:50:03][DEBUG]: == Working Agent: Content Writer
 [2024-09-07 10:50:03][INFO]: == Starting Task:     1. Use the content plan to craft a compelling blog post on Mastering the Parameters (Temperature, Top-K, Top-P) of Large Language Models based on the target audience's interests.
    2. Sections/Subtitles are properly named in an engaging manner.
    3. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing conclusion.
    4. Proofread for grammatical errors and alignment the common style us

## 💎💪🏼 Up for the challenge? 

Let's convert the code above (asychronous execution) into a Pyhton script that can be run with a click of a button.

---
---
<br>


# Extra: Agent with Ability to Run Code

The tool for executing Python code that comes with CrewAI requires additional setup (e.g., Docker), which may not be feasible in all environments.

We introduce a package called `open-interpreter` that allows you to run Python code, using the same interpreter where the current notebook/script is running.

- While this method is useful for apps with small scale code execution, it may not be suitable for large-scale applications due to putting the processing load on the same server that serves the app.

- For production-level code execution, it is recommended to use the `Code Interpreter` on separate server, or use more robust solution like the one provided the Cloud Service Provider ([Azure's example here](https://python.langchain.com/v0.2/docs/integrations/tools/azure_dynamic_sessions/)), or platforms that specialize in sandboxed code execution like [E2B](https://github.com/e2b-dev/e2b). 

- The standalone setup provide more security, as it isolates the code execution environment from the main server, and can be scaled independently.

<br>

> 📒 For more details on the package `open-interpreter`, vist their GitHub repo here https://github.com/OpenInterpreter/open-interpreter

In [ ]:
# To install the open-interpreter package
!pip install open-interpreter

In [ ]:
# This cell creates a custom tool that uses the open-interpreter package

from interpreter import interpreter
from langchain.agents import Tool

# Set the model to use for the interpreter
interpreter.llm.model = "gpt-4o-mini"

# Create the tool
tool_coding = Tool(
    name="Python Code Interpreter",
    func=interpreter.chat, # <-- This is the function that will be called when the tool is run. Note that there is no `()` at the end
    description="Useful for running Python3 Code",
)

💡 You will be prompted in VS Code to give permission for the package to execute the code (multiple times). 

he programs will seems to be stuck, but it is just waiting for your permission to execute the code.

In [16]:

# Creating Agents
agent_programmer = Agent(
    role="Programmer",
    goal="Write and execute code and share the output: {topic}",
    backstory="""You're the top-notch programmer who follows the best practices in Python coding.""",
    allow_delegation=False,
	verbose=True,
    tools=[tool_coding],
)

task_code = Task(
    description="""\
    1. Understand the user request on the requirements: {topic}.
    2. Write the code to meet the requirements. Proceed to next step only if the code is correct.
    3. Run the code and share the output.
    4. Stop iterating once the output is correct.""",


    expected_output="""\
    1. The output of the code execution.
    2. The code used to generate the output.
    3. The Explanation of the code and the output, for non-technical users.
    4. End of the conversation""",

    agent=agent_programmer,
)

# Creating the Crew
crew = Crew(
    agents=[agent_programmer],
    tasks=[task_code],
    verbose=True
)

# Running the Crew
result = crew.kickoff(inputs={"topic": "Write a for-loop that prints numbers from 1 to 10"})

2024-09-07 11:23:17,595 - 8660111168 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-09-07 11:23:17][DEBUG]: == Working Agent: Programmer
 [2024-09-07 11:23:17][INFO]: == Starting Task:     1. Understand the user request on the requirements: Write a for-loop that prints numbers from 1 to 10.
    2. Write the code to meet the requirements. Proceed to next step only if the code is correct.
    3. Run the code and share the output.
    4. Stop iterating once the output is correct.


> Entering new CrewAgentExecutor chain...
Thought: I need to write a for-loop in Python that prints numbers from 1 to 10. This is a simple task that involves using the `range` function and a loop to iterate through the numbers.

Action: Python Code Interpreter  
Action Input: {"type": "string", "value": "for i in range(1, 11):\n    print(i)"}
 

I encountered an error while trying to use the tool. This was the error: Too many arguments to single-input tool Python Code Interpreter.
                Consider using StructuredTool instead. Args: ['string', 'for i in range(1, 11):\n    print(

Open Interpreter will require approval before running code.

Use interpreter -y to bypass this.

Press CTRL-C to exit.

Output()

Output()

Output()

 

[{'role': 'assistant', 'type': 'code', 'format': 'python', 'content': 'for i in range(1, 11): print(i)'}, {'role': 'computer', 'type': 'console', 'format': 'output', 'content': '1\n2\n3\n4\n5\n6\n7\n8\n9\n10\n'}, {'role': 'assistant', 'type': 'message', 'content': "The code executed successfully and printed the numbers from 1 to 10.\n\nIf there's anything else you'd like to accomplish, please let me know!"}]

Thought: I have successfully written and executed the code that prints numbers from 1 to 10. The next step is to summarize the output and provide a clear explanation of the code.

Final Answer:
1. Output of the code execution:
```
1
2
3
4
5
6
7
8
9
10
```

2. Code used to generate the output:
```python
for i in range(1, 11):
    print(i)
```

3. Explanation of the code and the output:
   - The code uses a `for` loop to iterate over a sequence of numbers generated by the `range` function.
   - `range(1, 11)` generates numbers starting from 1 up to (but not including) 11, which r

---
---

# Scratching the Surface?

By the name "Getting Started", you got it right that this notebook only covers the basics of CrewAI.

There are many more advanced features and functionalities that you can explore in the CrewAI documentation.
